In [3]:
import sympy as sp
import numpy as np
from scipy import optimize
import pymc as pm
import pytensor
import pytensor.tensor as pt

# Simple CGE

## Define the model

In [24]:
def cobb_douglass(output: sp.Symbol, tfp: sp.Symbol, inputs: list[sp.Symbol], shares: list[sp.Symbol]):
    if len(inputs) != len(shares):
        if (len(shares) + 1) != len(inputs):
            raise ValueError('The length of the shares should len(inputs), or len(inputs) - 1')
        shares.append(1 - sum(shares))

    return output - tfp * sp.prod([x **  a for x, a in zip(inputs, shares)])

In [179]:
variables = Y, C, INC, Ld, Kd, r, P, Leon = sp.symbols('Y C INC Ld Kd r P Leon', positive=True, real=True)
parameters = Ls, Ks, A, alpha = sp.symbols('Ls Ks A alpha', positive=True, real=True)

# Numeraire
w = 1

equations = [
    cobb_douglass(Y, A, [Kd, Ld], [alpha]),
    alpha * P * Y - r * Kd,
    (1 - alpha) * P * Y - w * Ld,
    INC - w * Ls - r * Ks,
    P * C - INC,
    C - Y,
    Kd - Ks,
    Ld - Ls - Leon
]

resid = sum([eq ** 2 for eq in equations])
f_resid = sp.lambdify(variables + parameters, resid)

In [332]:
for eq in equations:
    display(eq)

-A*Kd**alpha*Ld**(1 - alpha) + Y

-Kd*r + P*Y*alpha

-Ld + P*Y*(1 - alpha)

INC - Ks*r - Ls

C*P - INC

C - Y

Kd - Ks

Ld - Leon - Ls

## Compute Jacobian and Hessian

In [333]:
jac = sp.Matrix([[resid.diff(x)] for x in variables])
hess = jac.jacobian(variables)

In [353]:
jac

Matrix([
[       -2*A*Kd**alpha*Ld**(1 - alpha) - 2*C + 2*P*alpha*(-Kd*r + P*Y*alpha) + 2*P*(1 - alpha)*(-Ld + P*Y*(1 - alpha)) + 4*Y],
[                                                                                                2*C + 2*P*(C*P - INC) - 2*Y],
[                                                                                             -2*C*P + 4*INC - 2*Ks*r - 2*Ls],
[-2*A*Kd**alpha*Ld**(1 - alpha)*(1 - alpha)*(-A*Kd**alpha*Ld**(1 - alpha) + Y)/Ld + 4*Ld - 2*Leon - 2*Ls - 2*P*Y*(1 - alpha)],
[         -2*A*Kd**alpha*Ld**(1 - alpha)*alpha*(-A*Kd**alpha*Ld**(1 - alpha) + Y)/Kd + 2*Kd - 2*Ks - 2*r*(-Kd*r + P*Y*alpha)],
[                                                                         -2*Kd*(-Kd*r + P*Y*alpha) - 2*Ks*(INC - Ks*r - Ls)],
[                                  2*C*(C*P - INC) + 2*Y*alpha*(-Kd*r + P*Y*alpha) + 2*Y*(1 - alpha)*(-Ld + P*Y*(1 - alpha))],
[                                                                                                     

In [354]:
hess

Matrix([
[                                                                2*P**2*alpha**2 + 2*P**2*(1 - alpha)**2 + 4,            -2,     0,                                                                                                                                                                              -2*A*Kd**alpha*Ld**(1 - alpha)*(1 - alpha)/Ld - 2*P*(1 - alpha),                                                                                                                                                                               -2*A*Kd**alpha*Ld**(1 - alpha)*alpha/Kd - 2*P*alpha*r,        -2*Kd*P*alpha, 2*P*Y*alpha**2 + 2*P*Y*(1 - alpha)**2 + 2*alpha*(-Kd*r + P*Y*alpha) + 2*(1 - alpha)*(-Ld + P*Y*(1 - alpha)),  0],
[                                                                                                         -2,    2*P**2 + 2,  -2*P,                                                                                                                             

In [334]:
f_jac = sp.lambdify(variables + parameters, jac)
f_hess = sp.lambdify(variables + parameters, hess)

## Compute feisible initial

In [355]:
def compute_system(initial_dict):
    calib_dict = initial_dict.copy()
    calib_dict[INC] = (w * Ls + r * Ks).subs(calib_dict)
    calib_dict[C] = (INC / P).subs(calib_dict)
    calib_dict[Y] = (C).subs(calib_dict)
    calib_dict[Kd] = (Ks).subs(calib_dict)
    calib_dict[Ld] = (Ls).subs(calib_dict)
    calib_dict[alpha] = ((r * Kd) / (Y * P)).subs(calib_dict).evalf()
    calib_dict[A] = sp.solve(equations[0].subs(calib_dict), A)[0].evalf()
    calib_dict[Leon] = (Ld - Ls).subs(calib_dict)

    return calib_dict


### Test inital feisible point

In [377]:
init_vals = {Ls:7000, Ks:4000, P:1, r:1}
state_1 = compute_system(init_vals)
state_1

{Ls: 7000,
 Ks: 4000,
 P: 1,
 r: 1,
 INC: 11000,
 C: 11000,
 Y: 11000,
 Kd: 4000,
 Ld: 7000,
 alpha: 0.363636363636364,
 A: 1.92607022425223,
 Leon: 0}

In [357]:
x0 = np.r_[[float(state_1[x]) for x in variables]]
param_vals = np.r_[[float(state_1[x]) for x in parameters]] 

f_resid_sp(x0, param_vals)

3.308722450212111e-24

## Simulate Labor Shock

Compute change in variables given a 10% increase to labor

### Non-Linear Solver

In [359]:
def sp_wrapper(f):

    def f_wrapped(x, args):
        if args is None:
            inputs = x
        else:
            inputs = np.r_[x, args]
        
        return f(*inputs).squeeze()

    return f_wrapped

f_resid_sp = sp_wrapper(f_resid)
f_jac_sp = sp_wrapper(f_jac)
f_hess_sp = sp_wrapper(f_hess)

state_2 = state_1.copy()
state_2[Ls] = 7700

x0 = np.r_[[float(state_2[x]) for x in variables]]
param_vals = np.r_[[float(state_2[x]) for x in parameters]] 
res = optimize.minimize(f_resid_sp, x0, jac=f_jac_sp, hess=f_hess_sp, args=param_vals, method='trust-krylov')

In [360]:
print(res.success)

True


In [369]:
optim_solution = dict(zip(variables, res.x))
optim_solution

{Y: 11687.819199172682,
 C: 11687.81919917287,
 INC: 12099.999999997124,
 Ld: 7699.999999997771,
 Kd: 4000.000000000683,
 r: 1.0999999999993255,
 P: 1.0352658433365507,
 Leon: -2.2285546786866145e-09}

In [379]:
deltas = {x: (optim_solution[x] / state_1[x] - 1) for x in variables}
deltas

{Y: 0.0625290181066074,
 C: 0.0625290181066245,
 INC: 0.0999999999997385,
 Ld: 0.0999999999996817,
 Kd: 1.70752301187349e-13,
 r: 0.09999999999932552,
 P: 0.035265843336550695,
 Leon: zoo}

### First-order taylor approximation

In [362]:
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

In [363]:
init_vals = Y0, C0, INC0, Ld0, Kd0, r0, P0, Leon0 = sp.symbols('Y_0 C_0 INC_0 Ld_0 Kd_0 r_0 P_0 Leon_0')
init_subs = dict(zip(variables, init_vals))

In [364]:
expanded_eqs = [Taylor_polynomial_sympy(eq, variables, init_vals, 1) for eq in equations]

In [365]:
system, bias = sp.linear_eq_to_matrix(expanded_eqs, variables)
Ab = sp.Matrix([[system, bias]])
A_rref, pivots = Ab.rref()

In [366]:
linear_solution = A_rref[:, -1]
f_linear = sp.lambdify(parameters + init_vals, linear_solution)

In [367]:
x0 = np.r_[[float(state_2[x]) for x in variables]]
param_vals = np.r_[[float(state_2[x]) for x in parameters]] 
taylor_solutions = dict(zip(variables, f_linear(*np.r_[param_vals, x0]).squeeze()))

In [368]:
taylor_solutions

{Y: 11700.0,
 C: 11700.0,
 INC: 12099.999999999998,
 Ld: 7699.999999999997,
 Kd: 4000.0,
 r: 1.0999999999999996,
 P: 1.036363636363636,
 Leon: -5.102040816326529e-12}

### Taylor Error

In [376]:
for k, v in optim_solution.items():
    v = float(v)
    tv = float(taylor_solutions[k])
    
    print(f'{k.name:<20}{(tv - v) / v :0.2%}')

Y                   0.10%
C                   0.10%
INC                 0.00%
Ld                  0.00%
Kd                  -0.00%
r                   0.00%
P                   0.11%
Leon                -99.77%
